In [4]:
import pandas as pd
import numpy as np
import os
from ctgan import CTGANSynthesizer
from ctgan import load_demo
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import make_scorer
from sdv.evaluation import evaluate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from time import time
from sklearn.utils.fixes import loguniform
from sklearn.svm import SVC

# Cognitive Workload Dataset

In [5]:
filenames = os.listdir(r'/Users/apostoloskalatzis/Downloads/ML project/cw/no_overlap/normalized')

In [6]:
filenames = os.listdir(r'/Users/apostoloskalatzis/Downloads/ML project/cw/no_overlap/normalized')

li = []
for filename in filenames:
    dataframe=pd.read_csv(os.path.join(r'/Users/apostoloskalatzis/Downloads/ML project/cw/no_overlap/normalized',filename))
    li.append(dataframe)

data = pd.concat(li, axis=0, ignore_index=True)

In [7]:
columns = [ 'HR_mean', 'HR_std', 'HR_min', 'HR_max', 'HRV_RMSSD',
       'HRV_MeanNN', 'HRV_SDNN', 'HRV_CVNN', 'HRV_CVSD', 'HRV_MedianNN',
       'HRV_MadNN', 'HRV_MCVNN', 'HRV_pNN50', 'HRV_pNN20', 'HRV_HF', 'HRV_VHF',
       'HRV_HFn', 'HRV_LFHF', 'HRV_SD1', 'HRV_SD2', 'HRV_SD1SD2', 'RSP_mean',
       'RSP_std', 'RSP_min', 'RSP_max', 'RRV_RMSSD', 'RRV_MeanBB', 'RRV_SDBB',
       'RRV_SD1', 'RRV_SD2', 'RRV_SD2SD1']

In [8]:
gp = data["ID"].values

In [9]:
data1= data[columns]
X=data1[columns]
y=data['label']

In [10]:
logo = LeaveOneGroupOut()
logo.get_n_splits(X, y, gp)

28

## Hyperparameter Tuning for Random Forest 

In [11]:
# GAN parameters
from sklearn.utils.fixes import loguniform
param_dist = {
    "max_depth": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    "max_features": ['auto', 'sqrt'],
    "bootstrap": [True, False],
    "min_samples_leaf": [1, 2, 4],
    "min_samples_split": [2, 5, 10],
    "n_estimators": [50,100, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}

n_iter_search = 15
random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_dist, n_iter=n_iter_search,verbose=5, cv=logo
)

In [12]:
# code taken from https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py
start = time()
random_search.fit(X, y, gp)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")
            
            

report(random_search.cv_results_)

/Users/apostoloskalatzis/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass groups=[ 4  4  4  4  4  4  5  5  5  5  5  5  7  7  7  7  7  7  6  6  6  6  6  6
 22 22 22 22 22 22 23 23 23 23 23 23 35 35 35 35 35 35 21 21 21 21 21 21
 20 20 20 20 20 20 34 34 34 34 34 30 30 30 30 30 24 24 24 24 24 31 31 31
 31 31 31 19 19 19 19 19 27 27 27 27 27 27 33 33 33 33 33 33 32 32 32 32
 32 32 26 26 26 26 26 17 17 17 17 17 17 16 16 16 16 16 16 28 28 28 28 28
 29 29 29 29 29 29 11 11 11 11 11 11 10 10 10 10 10 10 12 12 12 12 12 12
 13 13 13 13 13  8  8  8  8  8  8  9  9  9  9  9  9] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Fitting 28 folds for each of 15 candidates, totalling 420 fits
[CV 1/28] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time=   1.2s
[CV 2/28] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time=   1.1s
[CV 3/28] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time=   1.1s
[CV 4/28] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time=   1.2s
[CV 5/28] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time=   1.1s
[CV 6/28] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time=   1.1s
[CV 7/28] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_lea

[CV 2/28] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   0.4s
[CV 3/28] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   0.4s
[CV 4/28] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   0.4s
[CV 5/28] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   0.4s
[CV 6/28] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   0.4s
[CV 7/28] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time=   0.4s
[CV 8/28] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; tota

[CV 6/28] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 7/28] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 8/28] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 9/28] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 10/28] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 11/28] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 12/28] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV 13/28]

[CV 8/28] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 9/28] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 10/28] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 11/28] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 12/28] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 13/28] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=600; total time=   0.4s
[CV 14/28] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=600; total time=   0.4s
[

[CV 10/28] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s
[CV 11/28] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s
[CV 12/28] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s
[CV 13/28] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s
[CV 14/28] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s
[CV 15/28] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s
[CV 16/28] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.3s

[CV 12/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=2000; total time=   1.3s
[CV 13/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=2000; total time=   1.3s
[CV 14/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=2000; total time=   1.3s
[CV 15/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=2000; total time=   1.3s
[CV 16/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=2000; total time=   1.3s
[CV 17/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=2000; total time=   1.3s
[CV 18/28] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=2000; total time=

[CV 13/28] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=800; total time=   0.6s
[CV 14/28] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=800; total time=   0.6s
[CV 15/28] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=800; total time=   0.6s
[CV 16/28] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=800; total time=   0.6s
[CV 17/28] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=800; total time=   0.6s
[CV 18/28] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=800; total time=   0.6s
[CV 19/28] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=800; total time=   0.6s

[CV 15/28] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV 16/28] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV 17/28] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV 18/28] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV 19/28] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV 20/28] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV 21/28] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV 22

## Hyperparameter Tuning for Linear SVM 

In [13]:
param_dist = {
    "C": loguniform(1e0, 1e3),
    "gamma": loguniform(1e-4, 1e-3),
    "kernel": ["linear"],
    "class_weight": ["balanced", None],
}

n_iter_search = 15

random_search = RandomizedSearchCV(
    SVC(), param_distributions=param_dist, n_iter=n_iter_search, verbose=5, cv=logo
)

In [14]:
start = time()
random_search.fit(X, y, gp)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")


report(random_search.cv_results_)

/Users/apostoloskalatzis/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass groups=[ 4  4  4  4  4  4  5  5  5  5  5  5  7  7  7  7  7  7  6  6  6  6  6  6
 22 22 22 22 22 22 23 23 23 23 23 23 35 35 35 35 35 35 21 21 21 21 21 21
 20 20 20 20 20 20 34 34 34 34 34 30 30 30 30 30 24 24 24 24 24 31 31 31
 31 31 31 19 19 19 19 19 27 27 27 27 27 27 33 33 33 33 33 33 32 32 32 32
 32 32 26 26 26 26 26 17 17 17 17 17 17 16 16 16 16 16 16 28 28 28 28 28
 29 29 29 29 29 29 11 11 11 11 11 11 10 10 10 10 10 10 12 12 12 12 12 12
 13 13 13 13 13  8  8  8  8  8  8  9  9  9  9  9  9] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Fitting 28 folds for each of 15 candidates, totalling 420 fits
[CV 1/28] END C=25.296562040169754, class_weight=balanced, gamma=0.00010036395063055029, kernel=linear; total time=   1.3s
[CV 2/28] END C=25.296562040169754, class_weight=balanced, gamma=0.00010036395063055029, kernel=linear; total time=   0.0s
[CV 3/28] END C=25.296562040169754, class_weight=balanced, gamma=0.00010036395063055029, kernel=linear; total time=   0.0s
[CV 4/28] END C=25.296562040169754, class_weight=balanced, gamma=0.00010036395063055029, kernel=linear; total time=   0.0s
[CV 5/28] END C=25.296562040169754, class_weight=balanced, gamma=0.00010036395063055029, kernel=linear; total time=   0.0s
[CV 6/28] END C=25.296562040169754, class_weight=balanced, gamma=0.00010036395063055029, kernel=linear; total time=   0.0s
[CV 7/28] END C=25.296562040169754, class_weight=balanced, gamma=0.00010036395063055029, kernel=linear; total time=   0.0s
[CV 8/28] END C=25.296562040169754, class_weight=balanced, gamma=0.000100363

[CV 5/28] END C=41.317796194533834, class_weight=balanced, gamma=0.0004230485404470101, kernel=linear; total time=   0.0s
[CV 6/28] END C=41.317796194533834, class_weight=balanced, gamma=0.0004230485404470101, kernel=linear; total time=   0.0s
[CV 7/28] END C=41.317796194533834, class_weight=balanced, gamma=0.0004230485404470101, kernel=linear; total time=   0.0s
[CV 8/28] END C=41.317796194533834, class_weight=balanced, gamma=0.0004230485404470101, kernel=linear; total time=   0.0s
[CV 9/28] END C=41.317796194533834, class_weight=balanced, gamma=0.0004230485404470101, kernel=linear; total time=   0.0s
[CV 10/28] END C=41.317796194533834, class_weight=balanced, gamma=0.0004230485404470101, kernel=linear; total time=   0.0s
[CV 11/28] END C=41.317796194533834, class_weight=balanced, gamma=0.0004230485404470101, kernel=linear; total time=   0.0s
[CV 12/28] END C=41.317796194533834, class_weight=balanced, gamma=0.0004230485404470101, kernel=linear; total time=   0.0s
[CV 13/28] END C=41.3

[CV 18/28] END C=22.602254900956638, class_weight=None, gamma=0.0003872068296726566, kernel=linear; total time=   0.0s
[CV 19/28] END C=22.602254900956638, class_weight=None, gamma=0.0003872068296726566, kernel=linear; total time=   0.0s
[CV 20/28] END C=22.602254900956638, class_weight=None, gamma=0.0003872068296726566, kernel=linear; total time=   0.0s
[CV 21/28] END C=22.602254900956638, class_weight=None, gamma=0.0003872068296726566, kernel=linear; total time=   0.0s
[CV 22/28] END C=22.602254900956638, class_weight=None, gamma=0.0003872068296726566, kernel=linear; total time=   0.0s
[CV 23/28] END C=22.602254900956638, class_weight=None, gamma=0.0003872068296726566, kernel=linear; total time=   0.0s
[CV 24/28] END C=22.602254900956638, class_weight=None, gamma=0.0003872068296726566, kernel=linear; total time=   0.0s
[CV 25/28] END C=22.602254900956638, class_weight=None, gamma=0.0003872068296726566, kernel=linear; total time=   0.0s
[CV 26/28] END C=22.602254900956638, class_weigh

[CV 6/28] END C=18.513015831269822, class_weight=None, gamma=0.0008898462423908609, kernel=linear; total time=   0.0s
[CV 7/28] END C=18.513015831269822, class_weight=None, gamma=0.0008898462423908609, kernel=linear; total time=   0.0s
[CV 8/28] END C=18.513015831269822, class_weight=None, gamma=0.0008898462423908609, kernel=linear; total time=   0.0s
[CV 9/28] END C=18.513015831269822, class_weight=None, gamma=0.0008898462423908609, kernel=linear; total time=   0.0s
[CV 10/28] END C=18.513015831269822, class_weight=None, gamma=0.0008898462423908609, kernel=linear; total time=   0.0s
[CV 11/28] END C=18.513015831269822, class_weight=None, gamma=0.0008898462423908609, kernel=linear; total time=   0.0s
[CV 12/28] END C=18.513015831269822, class_weight=None, gamma=0.0008898462423908609, kernel=linear; total time=   0.0s
[CV 13/28] END C=18.513015831269822, class_weight=None, gamma=0.0008898462423908609, kernel=linear; total time=   0.0s
[CV 14/28] END C=18.513015831269822, class_weight=No

[CV 20/28] END C=488.1332081896867, class_weight=None, gamma=0.0009093736105512757, kernel=linear; total time=   0.1s
[CV 21/28] END C=488.1332081896867, class_weight=None, gamma=0.0009093736105512757, kernel=linear; total time=   0.1s
[CV 22/28] END C=488.1332081896867, class_weight=None, gamma=0.0009093736105512757, kernel=linear; total time=   0.1s
[CV 23/28] END C=488.1332081896867, class_weight=None, gamma=0.0009093736105512757, kernel=linear; total time=   0.1s
[CV 24/28] END C=488.1332081896867, class_weight=None, gamma=0.0009093736105512757, kernel=linear; total time=   0.1s
[CV 25/28] END C=488.1332081896867, class_weight=None, gamma=0.0009093736105512757, kernel=linear; total time=   0.1s
[CV 26/28] END C=488.1332081896867, class_weight=None, gamma=0.0009093736105512757, kernel=linear; total time=   0.1s
[CV 27/28] END C=488.1332081896867, class_weight=None, gamma=0.0009093736105512757, kernel=linear; total time=   0.3s
[CV 28/28] END C=488.1332081896867, class_weight=None, g

[CV 9/28] END C=86.2349229696245, class_weight=None, gamma=0.0002312787587843469, kernel=linear; total time=   0.0s
[CV 10/28] END C=86.2349229696245, class_weight=None, gamma=0.0002312787587843469, kernel=linear; total time=   0.0s
[CV 11/28] END C=86.2349229696245, class_weight=None, gamma=0.0002312787587843469, kernel=linear; total time=   0.0s
[CV 12/28] END C=86.2349229696245, class_weight=None, gamma=0.0002312787587843469, kernel=linear; total time=   0.0s
[CV 13/28] END C=86.2349229696245, class_weight=None, gamma=0.0002312787587843469, kernel=linear; total time=   0.0s
[CV 14/28] END C=86.2349229696245, class_weight=None, gamma=0.0002312787587843469, kernel=linear; total time=   0.0s
[CV 15/28] END C=86.2349229696245, class_weight=None, gamma=0.0002312787587843469, kernel=linear; total time=   0.0s
[CV 16/28] END C=86.2349229696245, class_weight=None, gamma=0.0002312787587843469, kernel=linear; total time=   0.1s
[CV 17/28] END C=86.2349229696245, class_weight=None, gamma=0.000

## Hyperparameter tuning for non Linear SVM

In [17]:
param_dist = {
    "C": loguniform(1e0, 1e3),
    "gamma": loguniform(1e-4, 1e-3),
    "kernel": ["rbf"],
    "class_weight": ["balanced", None],
}

n_iter_search = 15

random_search = RandomizedSearchCV(
    SVC(), param_distributions=param_dist, n_iter=n_iter_search, verbose=5, cv=logo
)

In [18]:
start = time()
random_search.fit(X, y, gp)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")


report(random_search.cv_results_)

/Users/apostoloskalatzis/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass groups=[ 4  4  4  4  4  4  5  5  5  5  5  5  7  7  7  7  7  7  6  6  6  6  6  6
 22 22 22 22 22 22 23 23 23 23 23 23 35 35 35 35 35 35 21 21 21 21 21 21
 20 20 20 20 20 20 34 34 34 34 34 30 30 30 30 30 24 24 24 24 24 31 31 31
 31 31 31 19 19 19 19 19 27 27 27 27 27 27 33 33 33 33 33 33 32 32 32 32
 32 32 26 26 26 26 26 17 17 17 17 17 17 16 16 16 16 16 16 28 28 28 28 28
 29 29 29 29 29 29 11 11 11 11 11 11 10 10 10 10 10 10 12 12 12 12 12 12
 13 13 13 13 13  8  8  8  8  8  8  9  9  9  9  9  9] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Fitting 28 folds for each of 15 candidates, totalling 420 fits
[CV 1/28] END C=333.0989989488361, class_weight=balanced, gamma=0.0002809513208766343, kernel=rbf; total time=   0.0s
[CV 2/28] END C=333.0989989488361, class_weight=balanced, gamma=0.0002809513208766343, kernel=rbf; total time=   0.0s
[CV 3/28] END C=333.0989989488361, class_weight=balanced, gamma=0.0002809513208766343, kernel=rbf; total time=   0.0s
[CV 4/28] END C=333.0989989488361, class_weight=balanced, gamma=0.0002809513208766343, kernel=rbf; total time=   0.0s
[CV 5/28] END C=333.0989989488361, class_weight=balanced, gamma=0.0002809513208766343, kernel=rbf; total time=   0.0s
[CV 6/28] END C=333.0989989488361, class_weight=balanced, gamma=0.0002809513208766343, kernel=rbf; total time=   0.0s
[CV 7/28] END C=333.0989989488361, class_weight=balanced, gamma=0.0002809513208766343, kernel=rbf; total time=   0.0s
[CV 8/28] END C=333.0989989488361, class_weight=balanced, gamma=0.0002809513208766343, kernel=rbf; total time= 

[CV 19/28] END C=643.0968885060895, class_weight=None, gamma=0.0004321224158520038, kernel=rbf; total time=   0.0s
[CV 20/28] END C=643.0968885060895, class_weight=None, gamma=0.0004321224158520038, kernel=rbf; total time=   0.0s
[CV 21/28] END C=643.0968885060895, class_weight=None, gamma=0.0004321224158520038, kernel=rbf; total time=   0.0s
[CV 22/28] END C=643.0968885060895, class_weight=None, gamma=0.0004321224158520038, kernel=rbf; total time=   0.0s
[CV 23/28] END C=643.0968885060895, class_weight=None, gamma=0.0004321224158520038, kernel=rbf; total time=   0.0s
[CV 24/28] END C=643.0968885060895, class_weight=None, gamma=0.0004321224158520038, kernel=rbf; total time=   0.0s
[CV 25/28] END C=643.0968885060895, class_weight=None, gamma=0.0004321224158520038, kernel=rbf; total time=   0.0s
[CV 26/28] END C=643.0968885060895, class_weight=None, gamma=0.0004321224158520038, kernel=rbf; total time=   0.0s
[CV 27/28] END C=643.0968885060895, class_weight=None, gamma=0.00043212241585200

[CV 19/28] END C=3.118695269402125, class_weight=balanced, gamma=0.00015161015482406483, kernel=rbf; total time=   0.0s
[CV 20/28] END C=3.118695269402125, class_weight=balanced, gamma=0.00015161015482406483, kernel=rbf; total time=   0.0s
[CV 21/28] END C=3.118695269402125, class_weight=balanced, gamma=0.00015161015482406483, kernel=rbf; total time=   0.0s
[CV 22/28] END C=3.118695269402125, class_weight=balanced, gamma=0.00015161015482406483, kernel=rbf; total time=   0.0s
[CV 23/28] END C=3.118695269402125, class_weight=balanced, gamma=0.00015161015482406483, kernel=rbf; total time=   0.0s
[CV 24/28] END C=3.118695269402125, class_weight=balanced, gamma=0.00015161015482406483, kernel=rbf; total time=   0.0s
[CV 25/28] END C=3.118695269402125, class_weight=balanced, gamma=0.00015161015482406483, kernel=rbf; total time=   0.0s
[CV 26/28] END C=3.118695269402125, class_weight=balanced, gamma=0.00015161015482406483, kernel=rbf; total time=   0.0s
[CV 27/28] END C=3.118695269402125, clas

[CV 17/28] END C=43.28148254420596, class_weight=None, gamma=0.000282194326666528, kernel=rbf; total time=   0.0s
[CV 18/28] END C=43.28148254420596, class_weight=None, gamma=0.000282194326666528, kernel=rbf; total time=   0.0s
[CV 19/28] END C=43.28148254420596, class_weight=None, gamma=0.000282194326666528, kernel=rbf; total time=   0.0s
[CV 20/28] END C=43.28148254420596, class_weight=None, gamma=0.000282194326666528, kernel=rbf; total time=   0.0s
[CV 21/28] END C=43.28148254420596, class_weight=None, gamma=0.000282194326666528, kernel=rbf; total time=   0.0s
[CV 22/28] END C=43.28148254420596, class_weight=None, gamma=0.000282194326666528, kernel=rbf; total time=   0.0s
[CV 23/28] END C=43.28148254420596, class_weight=None, gamma=0.000282194326666528, kernel=rbf; total time=   0.0s
[CV 24/28] END C=43.28148254420596, class_weight=None, gamma=0.000282194326666528, kernel=rbf; total time=   0.0s
[CV 25/28] END C=43.28148254420596, class_weight=None, gamma=0.000282194326666528, kerne

# Stress Dataset